# Page Protection Analysis
This is a rough analysis of how page protections (restrictions) changed after articles were posted to the Social Media Traffic Report (https://en.wikipedia.org/wiki/User:HostBot/Social_media_traffic_report). It is not meant to be fully robust (I would then pull a control sample of articles to compare to -- likely the articles that were just below the threshold to be posted to the report), but give a quick sense of the impact of the report.

In [129]:
spark

In [140]:
import gzip

import pandas as pd

## Load in data
TSV file of all of the unique page IDs that have shown up in the social media traffic report and the first date they appeared.

In [8]:
pageids_path = "../social-media-traffic-reports/data/pageids_firstdate.tsv"
pageids_name = 'smtr_pageids'
df = pd.read_csv(pageids_path, sep='\t', header=None)
df.columns = ['pageid', 'first_date']
spark.createDataFrame(df).createOrReplaceTempView(pageids_name)
df.head()

,pageid,first_date
0,933889,2020_04_15
1,565250,2020_05_15
2,2473996,2020_04_21
3,327694,2020_04_24
4,28344340,2020_03_28


## Get page restrictions data via dumps (coarse look)

In [61]:
# 1 March 2020 dump
# This is the closest dump snapshot to when the social media traffic report was launched
pids = set(df['pageid'])
page_restrictions_20200301 = {}
prefix = 'INSERT INTO `page_restrictions` VALUES '
postfix = ';\n'
with gzip.open('/mnt/data/xmldatadumps/public/enwiki/20200301/enwiki-20200301-page_restrictions.sql.gz', 'rt') as fin:
    for line in fin:
        if line.startswith(prefix):
            line = line[len(prefix):-len(postfix)].replace('NULL','None')
            pages = eval(line)
            for pr_page, pr_type, pr_level, _, _, pr_expiry, _ in pages:
                if pr_page in pids:
                    if pr_expiry != 'infinity' and int(pr_expiry) and pr_expiry < df[df['pageid'] == pr_page]['first_date'].values[0].replace('_', ''):
                        continue
                    if pr_page not in page_restrictions_20200301:
                        page_restrictions_20200301[pr_page] = {}
                    page_restrictions_20200301[pr_page][pr_type] = pr_level

In [ ]:
# 1 May 2020 dump
# This is the closest dump snapshot to the current day
pids = set(df['pageid'])
page_restrictions_20200501 = {}
prefix = 'INSERT INTO `page_restrictions` VALUES '
postfix = ';\n'
with gzip.open('/mnt/data/xmldatadumps/public/enwiki/20200501/enwiki-20200501-page_restrictions.sql.gz', 'rt') as fin:
    for line in fin:
        if line.startswith(prefix):
            line = line[len(prefix):-len(postfix)].replace('NULL','None')
            pages = eval(line)
            for pr_page, pr_type, pr_level, _, _, pr_expiry, _ in pages:
                if pr_page in pids:
                    if pr_expiry != 'infinity' and int(pr_expiry) and pr_expiry < df[df['pageid'] == pr_page]['first_date'].values[0].replace('_', ''):
                        continue
                    if pr_page not in page_restrictions_20200501:
                        page_restrictions_20200501[pr_page] = {}
                    page_restrictions_20200501[pr_page][pr_type] = pr_level

In [62]:
# edit/move restrictions around start of the pilot
at_start = pd.DataFrame(page_restrictions_20200301).T
for c in at_start:
    print(at_start[c].value_counts())
    print()

autoconfirmed        230
extendedconfirmed      7
sysop                  1
Name: edit, dtype: int64

sysop                181
autoconfirmed        118
extendedconfirmed      4
Name: move, dtype: int64



In [146]:
# edit/move restrictions around end of the pilot
at_end = pd.DataFrame(page_restrictions_20200501).T
for c in at_end:
    print(at_end[c].value_counts())
    print()

autoconfirmed        297
extendedconfirmed     13
sysop                  1
Name: edit, dtype: int64

sysop                186
autoconfirmed        145
extendedconfirmed      9
Name: move, dtype: int64



## Get page restrictions data via logs (finer-grained look)
Though this is still missing expirations, so it's actually hard to track what the page protections were right before an article was posted and in the weeks following

In [33]:
# get page restriction changes since March 1st dump
res = spark.sql("""
SELECT page_id,
       meta.dt as log_timestamp,
       page_restrictions
  FROM event.mediawiki_page_restrictions_change pr
  LEFT JOIN {0} pi
       ON (pr.page_id = pi.pageid)
 WHERE year = 2020 and month >= 3 and `database` = 'enwiki'
 ORDER by page_id, log_timestamp LIMIT 100000
 """.format(pageids_name))

In [34]:
res.show(n=100)

+-------+--------------------+--------------------+-----------+
|page_id|       log_timestamp|   page_restrictions|prior_state|
+-------+--------------------+--------------------+-----------+
|     12|2020-04-20T19:45:52Z|[edit -> autoconf...|         []|
|    656|2020-03-20T18:52:47Z|[edit -> autoconf...|         []|
|    803|2020-03-29T05:43:26Z|[edit -> autoconf...|         []|
|   1009|2020-04-12T14:41:17Z|[edit -> autoconf...|         []|
|   1130|2020-04-11T22:22:38Z|[edit -> autoconf...|         []|
|   1130|2020-04-19T04:32:09Z|[edit -> autoconf...|         []|
|   1175|2020-03-02T03:58:47Z|[edit -> autoconf...|         []|
|   1182|2020-05-07T22:12:09Z|[edit -> autoconf...|         []|
|   1348|2020-04-28T01:20:08Z|[edit -> autoconf...|         []|
|   1530|2020-05-04T22:19:30Z|[edit -> autoconf...|         []|
|   1541|2020-03-18T13:36:24Z|[edit -> autoconf...|         []|
|   1624|2020-03-09T01:04:46Z|[edit -> autoconf...|         []|
|   1653|2020-04-30T01:49:27Z|[edit -> a

In [35]:
page_restrictions_since_20200301 = res.toPandas()

In [111]:
def restrictions_at_first_post(row):
    # get best guess at restrictions right before when the article was posted
    pid = row['pageid']
    first_date = row['first_date'].replace('_', '')
    restrictions = {}
    if pid in page_restrictions_20200301:
        restrictions = page_restrictions_20200301[pid]
    updates = page_restrictions_since_20200301[page_restrictions_since_20200301['page_id'] == pid]
    for i in range(len(updates)):
        l = updates.iloc[i]
        ts = l['log_timestamp'].replace('-', '')
        ts = ts[:ts.index('T')]
        if ts > first_date:
            break
        for k,v in l['page_restrictions'].items():
            restrictions[k] = v
    if restrictions:
        return restrictions
    return None

def restrictions_after_first_post(row):
    # get best guess at restrictions after the article was posted
    # this should help fill in data post May 1st (last dump) and capture temporary protections
    # that had expired before the May 1st dump
    pid = row['pageid']
    first_date = row['first_date'].replace('_', '')
    restrictions = {}
    if pid in page_restrictions_20200301:
        restrictions = page_restrictions_20200301[pid]
    updates = page_restrictions_since_20200301[page_restrictions_since_20200301['page_id'] == pid]
    for i in range(len(updates)):
        l = updates.iloc[i]
        ts = l['log_timestamp'].replace('-', '')
        ts = ts[:ts.index('T')]
        for k,v in l['page_restrictions'].items():
            restrictions[k] = v
    if restrictions:
        return restrictions
    return None

In [115]:
df['at_start'] = df.apply(lambda x: restrictions_at_first_post(x), axis=1)
df['after_post'] = df.apply(lambda x: restrictions_after_first_post(x), axis=1)

In [119]:
df['at_start'].apply(lambda x: str(x)).value_counts()

None                                                          2874
{'edit': 'autoconfirmed', 'move': 'autoconfirmed'}             167
{'edit': 'autoconfirmed', 'move': 'sysop'}                      97
{'move': 'sysop'}                                               74
{'edit': 'autoconfirmed', 'move': ''}                           51
{'edit': 'autoconfirmed'}                                       25
{'move': 'sysop', 'edit': 'autoconfirmed'}                      11
{'edit': 'extendedconfirmed', 'move': 'extendedconfirmed'}      10
{'move': 'autoconfirmed'}                                        4
{'edit': 'sysop', 'move': 'sysop'}                               4
{'edit': 'extendedconfirmed', 'move': 'sysop'}                   3
{'move': 'autoconfirmed', 'edit': 'autoconfirmed'}               2
{'edit': '', 'move': 'sysop'}                                    1
{'edit': 'extendedconfirmed', 'move': ''}                        1
Name: at_start, dtype: int64

In [120]:
df['after_post'].apply(lambda x: str(x)).value_counts()

None                                                          2823
{'edit': 'autoconfirmed', 'move': 'autoconfirmed'}             191
{'edit': 'autoconfirmed', 'move': 'sysop'}                      99
{'edit': 'autoconfirmed', 'move': ''}                           75
{'move': 'sysop'}                                               74
{'edit': 'autoconfirmed'}                                       25
{'move': 'sysop', 'edit': 'autoconfirmed'}                      11
{'edit': 'extendedconfirmed', 'move': 'extendedconfirmed'}      10
{'move': 'autoconfirmed'}                                        4
{'edit': 'sysop', 'move': 'sysop'}                               4
{'edit': 'extendedconfirmed', 'move': 'sysop'}                   3
{'move': 'autoconfirmed', 'edit': 'autoconfirmed'}               2
{'edit': '', 'move': 'sysop'}                                    1
{'edit': 'autoconfirmed', 'move': 'extendedconfirmed'}           1
{'edit': 'extendedconfirmed', 'move': ''}                     

In [125]:
print("{0} pages.".format(len(df)))
protections_same = 0
protections_increased = 0
protections_decreased = 0
for pid in pids:
    at_start = df[df['pageid'] == pid]['at_start'].values[0]
    after_post = df[df['pageid'] == pid]['after_post'].values[0]
    if at_start is None and after_post is None:
        protections_same += 1
    elif at_start is None:
        protections_increased += 1
    elif after_post is None:
        protections_decreased += 1
    else:
        if len(at_start) == len(after_post):
            protections_same += 1
        elif len(at_start) > len(after_post):
            protections_decreased += 1
        else:
            protections_increased += 1
print("{0} ({1:.3f}) pages had same protections.".format(protections_same, protections_same / len(df)))
print("{0} ({1:.3f}) pages had protections increased.".format(protections_increased, protections_increased / len(df)))
print("{0} ({1:.3f}) pages had protections decreased.".format(protections_decreased, protections_decreased / len(df)))

3324 pages.
3273 (0.985) pages had same protections.
51 (0.015) pages had protections increased.
0 (0.000) pages had protections decreased.


In [145]:
print("{0} pages.".format(len(df)))
added = {}
removed = {}
no_change = 0
for pid in pids:
    at_start = df[df['pageid'] == pid]['at_start'].values[0]
    after_post = df[df['pageid'] == pid]['after_post'].values[0]
    if at_start == after_post:
        no_change += 1
    else:
        a = []
        r = []
        if at_start is None:
            at_start = {}
        if after_post is None:
            after_post = {}
        for p in at_start:
            if at_start[p] != after_post.get(p, None):
                r.append((p, at_start[p]))
        r = sorted(r)
        for p in after_post:
            if after_post[p] != at_start.get(p, None):
                a.append((p, after_post[p]))
        a = sorted(a)
        if r:
            removed[str(r)] = removed.get(str(r), 0) + 1
        if a:
            added[str(a)] = added.get(str(a), 0) + 1

print("{0} ({1:.3f}) pages had no change.".format(no_change, no_change / len(df)))
print("{0} ({1:.3f}) pages had the following additions: {2}".format(sum(added.values()), sum(added.values()) / len(df), added))
print("{0} ({1:.3f}) pages had the following removals: {2}".format(sum(removed.values()), sum(removed.values()) / len(df), removed))

3324 pages.
3260 (0.981) pages had no change.
64 (0.019) pages had the following additions: {"[('move', 'extendedconfirmed')]": 1, "[('move', 'sysop')]": 2, "[('edit', 'autoconfirmed'), ('move', 'autoconfirmed')]": 24, "[('move', '')]": 4, "[('move', 'autoconfirmed')]": 6, "[('edit', 'autoconfirmed'), ('move', '')]": 27}
13 (0.004) pages had the following removals: {"[('move', '')]": 7, "[('move', 'autoconfirmed')]": 6}
